In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.metrics import confusion_matrix 
import tensorflow as tf

In [3]:
train_dir='C:/Users/HP/dataset_train/Train_set'
test_dir='C:/Users/HP/dataset_test/Test_data'
val_dir='C:/Users/HP/dataset_val/dataset_val/val_data'

In [4]:
IMG_HEIGHT=224
IMG_WIDTH=224
BATCHSIZE=32


In [5]:
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator( 
    rescale=1.0/255,
    rotation_range=20,
    brightness_range=(1.2,1.5),
    horizontal_flip=True)
test_datagen=tf.keras.preprocessing.image.ImageDataGenerator( rescale=1.0/255)
val_datagen=tf.keras.preprocessing.image.ImageDataGenerator( rescale=1.0/255)

In [6]:
train_data=train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT,IMG_WIDTH),
    class_mode='binary',
    batch_size=BATCHSIZE)

test_data=test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT,IMG_WIDTH),
    class_mode='binary',
    batch_size=BATCHSIZE)

val_data=val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT,IMG_WIDTH),
    class_mode='binary',
batch_size=BATCHSIZE)

Found 1445 images belonging to 5 classes.
Found 442 images belonging to 5 classes.
Found 442 images belonging to 5 classes.


In [7]:
mobilenet=tf.keras.applications.MobileNetV2(
input_shape=(IMG_HEIGHT,IMG_WIDTH,3),
include_top=False,
weights='imagenet',
pooling='avg')
mobilenet.trainable=False

In [8]:

inputs=tf.keras.Input(shape=(IMG_HEIGHT,IMG_WIDTH,3))
pretrained_model=mobilenet(inputs,training=False)
dense=tf.keras.layers.Dense(1024,activation='relu')(pretrained_model)
outputs=tf.keras.layers.Dense(1,activation='sigmoid')(dense)

model=tf.keras.Model(inputs,outputs)
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 1280)             2257984   
 ional)                                                          
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dense_1 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 3,570,753
Trainable params: 1,312,769
Non-trainable params: 2,257,984
_________________________________________________________________
None


In [ ]:
EPOCHS=50
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy',tf.keras.metrics.AUC(name='auc')
            ]
)

history=model.fit(
train_data,
validation_data=val_data,
batch_size=BATCHSIZE,
epochs=EPOCHS,
callbacks=[tf.keras.callbacks.EarlyStopping(
monitor='val_loss',
patience=3,
restore_best_weights=True)
          ]
)

Epoch 1/50
46/46 [==============================] - 104s 2s/step - loss: -174.8008 - accuracy: 0.2893 - auc: 0.4952 - val_loss: -384.7964 - val_accuracy: 0.4548 - val_auc: 0.5000
Epoch 2/50
46/46 [==============================] - 98s 2s/step - loss: -1006.0706 - accuracy: 0.2941 - auc: 0.5000 - val_loss: -1454.5215 - val_accuracy: 0.4548 - val_auc: 0.5000
Epoch 3/50
46/46 [==============================] - 97s 2s/step - loss: -2747.1987 - accuracy: 0.2941 - auc: 0.5000 - val_loss: -3201.6221 - val_accuracy: 0.4548 - val_auc: 0.5000
Epoch 4/50
46/46 [==============================] - 96s 2s/step - loss: -5536.6138 - accuracy: 0.2941 - auc: 0.5000 - val_loss: -5960.7974 - val_accuracy: 0.4548 - val_auc: 0.5000
Epoch 5/50
46/46 [==============================] - 96s 2s/step - loss: -9641.5098 - accuracy: 0.2941 - auc: 0.5000 - val_loss: -9810.4453 - val_accuracy: 0.4548 - val_auc: 0.5000
Epoch 6/50
46/46 [==============================] - 97s 2s/step - loss: -14704.0859 - accuracy: 0.294

In [ ]:
true_labels=test_data.labels
predict_labels=np.squeeze(np.array(model.predict(test_data) >=.5,dtype=np.int))
cm=confusion_matrix(true_labels,predict_labels)


In [ ]:
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True,fmt='g', vmin=0, cmap='mako',cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.xticks(np.arange(5)*.5,['Normal','Doubtful','Mild','Moderate','Severe'],fontsize = 16)
plt.yticks(np.arange(5)*.5,['Normal','Doubtful','Mild','Moderate','Severe'],fontsize = 16)
plt.show()

In [ ]:
results = model.evaluate(test_data,verbose=0)

accuracy=results[1]
auc=results[2]

tn,fp,fn,tp=cm.ravel()
precision = tp/(tp+fp)
recall=tp/(tp+fn)

print("Accuracy:(.2f)",format(accuracy))
print("Auc:(.2f)",format(auc))
print("Precision:(.2f)",format(precision))
print("Recall:(.2f)",format(recall))